# Preparação do ambiente Spark

In [ ]:
!pip -qq install pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName("Classificação com Spark")\
    .getOrCreate()

# Importação dos dados

In [4]:
!mkdir assets
! wget -qq -O ./assets/dataset.csv https://github.com/kamillafsilva/preditor_churn/blob/main/assets/dataset.csv?raw=true

In [5]:
raw = spark.read.csv('./assets/dataset.csv', sep=',', header = True, inferSchema = True)

In [6]:
raw.show(5, truncate = False)

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|id |Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet   |SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio|MetodoPagamento |MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|0  |Nao  |0         |Sim    |Nao        |1              |Nao         |SemServicoTelefonico    |DSL        |Nao            |Sim         |Nao              |Nao           |Nao    |Nao            |Mensalmente |Sim         |BoletoEletronico|29.85  

In [7]:
raw.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



# Preparando os dados

## Dados faltantes

In [8]:
from pyspark.sql.types import IntegerType, DoubleType, StringType
from pyspark.sql import functions as f

In [9]:
num = [f.name for f in raw.schema.fields if not isinstance(f.dataType, StringType)] #campos numéricos
num.remove('id')
num.remove('Mais65anos')

In [10]:
raw\
  .select([f.count(f.when(f.isnull(c) | f.isnan(c), 1)).alias(c) for c in num])\
  .show()

+---------------+-------------+
|MesesDeContrato|MesesCobrados|
+---------------+-------------+
|              0|            0|
+---------------+-------------+



In [11]:
cat = [f.name for f in raw.schema.fields if isinstance(f.dataType, StringType)] #campos categóricos

In [12]:
raw\
  .select([f.count(f.when(f.isnull(c) | (f.col(c) == ''), 1)).alias(c) for c in cat])\
  .show()

+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+---------------+
|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio|MetodoPagamento|
+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+---------------+
|    0|      0|          0|           0|                       0|       0|              0|           0|                0|             0|      0|              0|           0|           0|              0|
+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--

# Medidas de resumo

In [13]:
raw.select(num).summary().show()

+-------+------------------+------------------+
|summary|   MesesDeContrato|     MesesCobrados|
+-------+------------------+------------------+
|  count|             10348|             10348|
|   mean|27.518554310011595| 67.72477324116379|
| stddev| 24.09778298180764|28.859250112596182|
|    min|                 0|             18.25|
|    25%|                 5|             45.35|
|    50%|                21| 74.57758818204512|
|    75%|                49|              90.8|
|    max|                72|            118.75|
+-------+------------------+------------------+



In [14]:
raw\
  .select([f.countDistinct(f.col(c)).alias(c) for c in cat])\
  .show()

+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+---------------+
|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio|MetodoPagamento|
+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+---------------+
|    2|      2|          2|           2|                       3|       3|              3|           3|                3|             3|      3|              3|           3|           2|              4|
+-----+-------+-----------+------------+------------------------+--------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--

In [15]:
raw \
  .select('Churn')\
  .groupBy('Churn')\
  .count()\
  .show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [16]:
raw = raw.withColumnRenamed('Churn', 'target')

In [17]:
raw = raw.withColumn('target', f.when(f.lower(raw.target) == "sim", 1).otherwise(0).alias('target'))

In [18]:
raw \
  .groupBy('target')\
  .count()\
  .show()

+------+-----+
|target|count|
+------+-----+
|     1| 5174|
|     0| 5174|
+------+-----+



In [19]:
raw.write.parquet(path = './assets/processed')

# Construção do modelo

## Pré-processamento dos dados

In [20]:
processed = spark.read.parquet('./assets/processed')

In [21]:
processed.show(5)

+---+------+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
| id|target|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+------+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|  0|     0|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|            Nao|         Sim|              Nao|           Nao|    Nao|            Nao| Mensalmente|         Sim|BoletoEletronico|   

In [22]:
processed.printSchema()

root
 |-- id: integer (nullable = true)
 |-- target: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [23]:
num = [f.name for f in raw.schema.fields if not isinstance(f.dataType, StringType)] #campos numéricos
num.remove('id')

cat = [f.name for f in raw.schema.fields if isinstance(f.dataType, StringType)] #campos categóricos

In [24]:
treino, teste = processed.randomSplit([0.7, 0.3], seed = 42)

### OneHot Encoding, Vetorização e Escalonamento

In [25]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

In [26]:
features = num.copy()
features.remove('target')

In [27]:
indexer = StringIndexer(inputCols=cat, outputCols=['{}_indexed'.format(a) for a in cat])
encoder = OneHotEncoder(inputCols=indexer.getOutputCols(), outputCols=['{}_onehot'.format(a) for a in cat])
assembler = VectorAssembler(inputCols = features + encoder.getOutputCols(), outputCol = 'features')
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [28]:
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler])
pipeline_fitted = pipeline.fit(treino)

In [29]:
treino_scaled = pipeline_fitted.transform(treino).select('features_scaled', 'target')

In [30]:
treino_scaled.printSchema()

root
 |-- features_scaled: vector (nullable = true)
 |-- target: integer (nullable = true)



## Treinando modelos de Classificação

## Regressão Logística

In [31]:
from pyspark.ml.classification import LogisticRegression

In [32]:
logReg = LogisticRegression(featuresCol = 'features_scaled', labelCol='target')
logReg_fitted = logReg.fit(treino_scaled)

## Árvore de Decisão

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

In [34]:
tree = DecisionTreeClassifier(featuresCol = 'features_scaled', labelCol='target', seed = 42)
tree_fitted = tree.fit(treino_scaled)

## XGBoost

In [35]:
from xgboost.spark import SparkXGBClassifier

In [36]:
xgb_params = {'max_depth': 6, 'min_child_weight': 1, 'gamma': 0}

In [37]:
xgb = SparkXGBClassifier(features_col = 'features_scaled', label_col='target', seed = 42, verbosity = 0, **xgb_params)
xgb_fitted = xgb.fit(treino_scaled)

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'gamma': 0, 'max_depth': 6, 'min_child_weight': 1, 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!


## Avaliação dos modelos

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pandas as pd

In [39]:
def classification_report(model, data):
  label = model.getLabelCol()

  acc_evaluator = MulticlassClassificationEvaluator(labelCol = label, predictionCol="prediction",metricName=  'accuracy')
  f1_evaluator = MulticlassClassificationEvaluator(labelCol = label, predictionCol="prediction",metricName=  'f1')
  prec_evaluator = MulticlassClassificationEvaluator(labelCol = label, predictionCol="prediction",metricName=  'precisionByLabel')
  rec_evaluator = MulticlassClassificationEvaluator(labelCol = label, predictionCol="prediction",metricName=  'recallByLabel')

  preds = model.transform(data)

  return [preds, {'acc':acc_evaluator.evaluate(preds), 'f1':f1_evaluator.evaluate(preds), 'prec':prec_evaluator.evaluate(preds), 'rec':rec_evaluator.evaluate(preds)}]

In [40]:
def getConfusionMatrix(model, data):
  label = model.getLabelCol()
  preds = model.transform(data)
  preds_and_labels =preds.select(['prediction',label]).withColumn('label', f.col(label).cast(FloatType())).orderBy('prediction')
  preds_and_labels = preds_and_labels.select(['prediction', 'label'])
  metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
  return metrics.confusionMatrix().toArray()

In [41]:
models = [logReg_fitted, tree_fitted, xgb_fitted]
results_treino = []

for model in models:
  model_report = classification_report(model, treino_scaled)[1]
  results_treino.append(model_report)

In [42]:
pd.DataFrame(results_treino, index = ['logReg', 'tree', 'xgb'])

,acc,f1,prec,rec
logReg,0.786609,0.786450,0.807316,0.757158
tree,0.790147,0.790074,0.805603,0.769044
xgb,0.935901,0.935850,0.966234,0.904376


## Avaliando o modelo no teste

In [45]:
teste_scaled = pipeline_fitted.transform(teste)

In [46]:
results_teste = []

for model in models:
  model_report = classification_report(model, teste_scaled)[1]
  results_teste.append(model_report)

In [47]:
pd.DataFrame(results_teste, index = ['logReg', 'tree', 'xgb'])

,acc,f1,prec,rec
logReg,0.788333,0.788197,0.794926,0.766304
tree,0.792000,0.791940,0.794036,0.777853
xgb,0.817000,0.816945,0.820264,0.802989


## Ajuste de hiperparâmetro com `hyperopt`

In [52]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.early_stop import no_progress_loss

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [58]:
def prCrossValidator(params):
  estimator = SparkXGBClassifier(features_col = 'features_scaled', label_col='target', seed = 42, verbosity = 0)
  evaluator = BinaryClassificationEvaluator(labelCol = 'target', metricName = 'areaUnderPR')

  grid = ParamGridBuilder()\
  .addGrid(estimator.max_depth, [int(params['max_depth'])])\
  .addGrid(estimator.min_child_weight, [params['min_child_weight']])\
  .addGrid(estimator.gamma, [params['gamma']])\
  .build()

  cv = CrossValidator(
    estimator = estimator,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    numFolds=5,
    seed=42
  )

  cv_fitted = cv.fit(treino_scaled)
  pr_auc = cv_fitted.avgMetrics[0]

  return pr_auc

In [61]:
best = 0
pr = []

In [55]:
def objective_function(search_space):
    global best
    pr_auc = prCrossValidator(search_space)
    if pr_auc > best:
        best = pr_auc
        pr.append(search_space)
    return {"loss": -pr_auc, "status": STATUS_OK}

In [60]:
search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                'gamma': hp.uniform('gamma', 0, 0.5)
                }
trials = Trials()

In [62]:
best_hyperparam = fmin(
    fn = objective_function,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 40,
    trials = trials,
    early_stop_fn = no_progress_loss(15)
    )

  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.04651054809910343, 'max_depth': 6, 'min_child_weight': 19.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.04651054809910343, 'max_depth': 6, 'min_child_weight': 19.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.04651054809910343, 'max_depth': 6, 'min_child_weight': 19.0, 'objecti

  2%|▎         | 1/40 [00:29<19:24, 29.85s/trial, best loss: -0.870264108549154]

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.17873822506832326, 'max_depth': 12, 'min_child_weight': 15.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.17873822506832326, 'max_depth': 12, 'min_child_weight': 15.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.17873822506832326, 'max_depth': 12, 'min_child_weight': 15.0, 'obje

  5%|▌         | 2/40 [00:59<18:51, 29.77s/trial, best loss: -0.8736838584659401]

...

 95%|█████████▌| 38/40 [18:18<00:57, 28.61s/trial, best loss: -0.8834443662700473]

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.15816204773857778, 'max_depth': 15, 'min_child_weight': 3.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.15816204773857778, 'max_depth': 15, 'min_child_weight': 3.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.15816204773857778, 'max_depth': 15, 'min_child_weight': 3.0, 'objecti

 98%|█████████▊| 39/40 [18:46<00:28, 28.39s/trial, best loss: -0.8834443662700473]

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.2409241061866138, 'max_depth': 13, 'min_child_weight': 5.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.2409241061866138, 'max_depth': 13, 'min_child_weight': 5.0, 'objective': 'binary:logistic', 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!
INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'device': 'cpu', 'gamma': 0.2409241061866138, 'max_depth': 13, 'min_child_weight': 5.0, 'objective'

100%|██████████| 40/40 [19:14<00:00, 28.87s/trial, best loss: -0.8834443662700473]


In [63]:
print("Best: {}".format(best_hyperparam))

Best: {'gamma': 0.43449993329757747, 'max_depth': 12.0, 'min_child_weight': 1.0}


In [66]:
xgb_params = {'max_depth': int(best_hyperparam['max_depth']), 'min_child_weight': 1, 'gamma': 0}
xgb_hopt = SparkXGBClassifier(features_col = 'features_scaled', label_col='target', seed = 42, verbosity = 0, **xgb_params)

In [67]:
xgb_hopt_fitted = xgb_hopt.fit(treino_scaled)

INFO:XGBoost-PySpark:Running xgboost-2.0.3 on 1 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'gamma': 0, 'max_depth': 12, 'min_child_weight': 1, 'verbosity': 0, 'seed': 42, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!


In [70]:
models = [logReg_fitted, tree_fitted, xgb_fitted, xgb_hopt_fitted]
results_treino = []

for model in models:
  model_report = classification_report(model, treino_scaled)[1]
  results_treino.append(model_report)

In [71]:
pd.DataFrame(results_treino, index = ['logReg', 'tree', 'xgb', 'xgb_hopt'])

,acc,f1,prec,rec
logReg,0.786609,0.786450,0.807316,0.757158
tree,0.790147,0.790074,0.805603,0.769044
xgb,0.935901,0.935850,0.966234,0.904376
xgb_hopt,0.988977,0.988976,0.997800,0.980281


In [72]:
models = [logReg_fitted, tree_fitted, xgb_fitted, xgb_hopt_fitted]
results_teste = []

for model in models:
  model_report = classification_report(model, teste_scaled)[1]
  results_teste.append(model_report)

In [73]:
pd.DataFrame(results_teste, index = ['logReg', 'tree', 'xgb', 'xgb_hopt'])

,acc,f1,prec,rec
logReg,0.788333,0.788197,0.794926,0.766304
tree,0.792000,0.791940,0.794036,0.777853
xgb,0.817000,0.816945,0.820264,0.802989
xgb_hopt,0.826333,0.826309,0.826804,0.817255
